In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import os
import json
from scipy.stats import t
from datetime import datetime, timedelta


In [ ]:
path = 'C:/Users/Janet/OneDrive - The University of Chicago/Data_policy/final-project-janet'

file_name = 'output.csv'
file_path = os.path.join(path, file_name)
df = pd.read_csv(file_path)

In [ ]:


event_date = datetime(2018, 6, 28) # date of CCPA being signed
estimation_window_start = event_date - timedelta(days=365)
estimation_window_end = event_date - timedelta(days=30)
event_window_start = event_date - timedelta(days=10)
event_window_end = event_date + timedelta(days=10)
stock_list = df['ticker'].unique()
# stock_list = ['ATUS', 'AFTM', 'ATUS', 'AAPL', 'ARRY']  # List of stocks to analyze

combined_data = pd.DataFrame()

for ticker in stock_list:
    try:
        stock = yf.Ticker(ticker)
        
        # Fetch historical stock data using yf.download
        historical_data_adjClose = yf.download(ticker, start=estimation_window_start, end=event_window_end) #retrieved dataframe that include adj close column
        if historical_data_adjClose.empty:
            print(f"No data for {ticker} using yf.download for specified dates.")
            continue

        # Fetch historical stock data using stock.history
        historical_data_dividend = stock.history(start=estimation_window_start, end=event_window_end) #retrieved dataframe that include dividend column
        if historical_data_dividend.empty:
            print(f"No data for {ticker} using stock.history for specified dates.")
            continue

        # Convert both DataFrames to tz-naive (if they are tz-aware)
        historical_data_adjClose.index = historical_data_adjClose.index.tz_localize(None)
        historical_data_dividend.index = historical_data_dividend.index.tz_localize(None)

        # Merge the two dataframes on their index since both are indexed by date when retrieved
        historical_data = pd.merge(historical_data_adjClose, historical_data_dividend[['Dividends']], left_index=True, right_index=True, how='left')

        historical_data['ticker'] = ticker

        info = stock.info
        market_cap = info.get('marketCap', 'N/A')
        historical_data['marketCap'] = market_cap

        # Append this stock's data to the combined DataFrame
        combined_data = pd.concat([combined_data, historical_data])
    except Exception as e:
        print(f"Error processing {ticker}: {e}")


In [84]:
market_index_ticker = '^GSPC'  # Example: S&P 500
market_data = yf.download(market_index_ticker, start=estimation_window_start, end=event_window_end)
market_data = market_data[['Adj Close']].rename(columns={'Adj Close': 'market_adj_close'})
combined_data = combined_data.rename(columns={
    'Adj Close': 'stock_adj_close',
    'Volume': 'stock_volume',
    'Open': 'stock_open',
    'High': 'stock_high',
    'Low': 'stock_low',
    'Close': 'stock_close',
    'Dividends':'dividends',
    'marketCap':'stock_market_cap'
})
combined_data = combined_data.rename_axis('date')
market_data = market_data.rename_axis('date')
merged_data = pd.merge(combined_data, market_data, left_index=True, right_index=True, how='left')
# merged_data = pd.merge(historical_data_adjClose, historical_data_dividend[['Dividends']], left_index=True, right_index=True, how='left')


[*********************100%%**********************]  1 of 1 completed

In [85]:


unique_ticker_count = combined_data['ticker'].nunique()
print(unique_ticker_count)
print(combined_data['dividends'].nunique())
print(combined_data['stock_volume'].nunique())

258
98
19666


In [86]:
sp500_path = r'C:\Users\Janet\OneDrive - The University of Chicago\Data_policy\final-project-janet\data'
json_file_name = 'sp500_constituents.json'
json_file_path = os.path.join(sp500_path, json_file_name)
with open(json_file_path, 'r') as file:
    sp500_data = json.load(file)
sp500_tickers = sp500_data.get("2018/06/20", []) #extract sp500 constitutes list at the most recent date of event_date (2018/6/28)
merged_data['sp500'] = merged_data['ticker'].apply(lambda x: 1 if x in sp500_tickers else 0)

merged_data.to_csv('combined_stock_data.csv')

In [88]:
sp500_counts = merged_data['sp500'].value_counts()
count_sp500_is_1 = sp500_counts.get(1, 0)
print("Number of instances where sp500 equals 1:", count_sp500_is_1)

Number of instances where sp500 equals 1: 3870
